In [1]:
import json
import cv2
import os 
def bbox_actual_coco(folder_with_images, coco_annotations_file, result_folder):
    if not os.path.exists(folder_with_images):
        raise FileNotFoundError(folder_with_images)
    if not os.path.exists(coco_annotations_file):
        raise FileNotFoundError(coco_annotations_file)
    with open(coco_annotations_file, "r") as f:
        coco_annotations = json.load(f)
    os.makedirs(result_folder, exist_ok=True)
    filename_to_id = {image_info['file_name']: image_info['id'] for image_info in coco_annotations['images']}
    result_actual = []
    for image_filename in os.listdir(folder_with_images):
        image_path = os.path.join(folder_with_images, image_filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image is not found: {image_path}")
            continue
        image_id = filename_to_id.get(image_filename)
        if image_id is None:
            print(f"No annotation for this image: {image_filename}")
            continue
        image_annotations = [annotation for annotation in coco_annotations["annotations"] if annotation["image_id"] == image_id]
        for annotation in image_annotations:
            category_id = annotation["category_id"]
            category_info = next((category for category in coco_annotations["categories"] if category["id"] == category_id), None)
            category_name = category_info["name"]
            bbox = annotation["bbox"]
            x, y, width, height = bbox
            result_actual.append({'file_name': image_filename, 
            'category': category_name, 
            'bbox':bbox})
            cv2.rectangle(image, (int(x), int(y)), (int(x + width), int(y + height)), (0, 255, 0), 2)
            cv2.putText(image, category_name, (int(x), int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        result_folder_path = os.path.join(result_folder, image_filename)
        cv2.imwrite(result_folder_path, image)
    return result_actual    
folder_with_images = '/data/datasets/model_validation/val_yolo/images'
coco_annotations_file = '/data/datasets/model_validation/coco/annotations/instances_default.json'
bbox_actual_coco(folder_with_images, coco_annotations_file, 'result_folder')

[{'file_name': 'frame_001025.PNG',
  'category': 'person',
  'bbox': [637.0, 380.0, 58.0, 151.0]},
 {'file_name': 'frame_001025.PNG',
  'category': 'person',
  'bbox': [155.0, 413.0, 55.0, 114.0]},
 {'file_name': 'frame_001025.PNG',
  'category': 'person',
  'bbox': [161.29, 372.39, 20.0, 62.0]},
 {'file_name': 'frame_002391.PNG',
  'category': 'person',
  'bbox': [924.48, 369.18, 21.17, 49.5]},
 {'file_name': 'frame_002391.PNG',
  'category': 'person',
  'bbox': [499.73, 380.3, 25.35, 35.27]},
 {'file_name': 'frame_002391.PNG',
  'category': 'person',
  'bbox': [539.0, 366.0, 20.0, 45.0]},
 {'file_name': 'frame_002391.PNG',
  'category': 'person',
  'bbox': [403.0, 294.0, 29.0, 22.0]},
 {'file_name': 'frame_002391.PNG',
  'category': 'person',
  'bbox': [335.47, 278.21, 11.0, 25.0]},
 {'file_name': 'frame_002391.PNG',
  'category': 'person',
  'bbox': [332.66, 276.02, 9.6, 20.7]},
 {'file_name': 'frame_002391.PNG',
  'category': 'person',
  'bbox': [822.32, 200.44, 14.0, 29.0]},
 {'fi

In [2]:
from ultralytics import YOLO
model = YOLO("yolov8n")
from YOLOv8_predict import extract_detections_pt
def bbox_coord_predict(image_folder_with_actual):
    result_predict = []
    image_files = [file for file in os.listdir(image_folder_with_actual) if file.endswith(('.PNG', '.png'))]
    for image_name in image_files:
        image_path = os.path.join(image_folder_with_actual, image_name)
        img = cv2.imread(image_path)
        if img is None:
            continue
        predicted_data = model(img)
        for pred in predicted_data:
            predict_annot = extract_detections_pt(pred)
            for k in predict_annot:
                label = k['label']
                score = k['score']
                box = k['ltrb']
                x_min, y_min, x_max, y_max = box
                result_predict.append({
                    'file_name': image_name,
                    'category': label,
                    'conf_score': round(score, 2),
                    'bbox': box })
                cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 2)
                cv2.putText(img,f'{label}({score:2f})', (int(x_min), int(y_min) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            result_folder_path = os.path.join(image_folder_with_actual, image_name)
            cv2.imwrite(result_folder_path, img)
    return result_predict
 
image_folder_with_actual = '/data/datasets/validator/result_folder'
bbox_coord_predict(image_folder_with_actual)


0: 640x640 2 persons, 1 tie, 36.8ms
1: 640x640 4 persons, 1 bus, 36.8ms
Speed: 2.3ms preprocess, 36.8ms inference, 126.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 74.9ms
Speed: 0.8ms preprocess, 74.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3.8ms
Speed: 0.9ms preprocess, 3.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 3.8ms
Speed: 1.1ms preprocess, 3

[{'file_name': 'frame_002391.PNG',
  'category': 'person',
  'conf_score': 0.29,
  'bbox': [314, 29, 340, 93]},
 {'file_name': 'frame_001543.PNG',
  'category': 'person',
  'conf_score': 0.52,
  'bbox': [962, 380, 1022, 534]},
 {'file_name': 'frame_000069.PNG',
  'category': 'person',
  'conf_score': 0.29,
  'bbox': [936, 379, 998, 528]},
 {'file_name': 'frame_001255.PNG',
  'category': 'traffic light',
  'conf_score': 0.31,
  'bbox': [658, 377, 718, 523]},
 {'file_name': 'frame_000400.PNG',
  'category': 'traffic light',
  'conf_score': 0.34,
  'bbox': [666, 351, 731, 523]},
 {'file_name': 'frame_001414.PNG',
  'category': 'person',
  'conf_score': 0.63,
  'bbox': [826, 380, 879, 530]},
 {'file_name': 'frame_000629.PNG',
  'category': 'fire hydrant',
  'conf_score': 0.36,
  'bbox': [699, 384, 748, 473]},
 {'file_name': 'frame_000903.PNG',
  'category': 'traffic light',
  'conf_score': 0.47,
  'bbox': [662, 364, 732, 531]},
 {'file_name': 'frame_001314.PNG',
  'category': 'traffic ligh

In [3]:
def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    #print(f'box1:{box1}')
    x2_min, y2_min, x2_max, y2_max = box2
    #print(f'box2:{box2}')
    #coordinates of intersection rectangle
    x1 = max(x1_min, x2_min)
    y1 = max(y1_min, y2_min)
    x2 = min(x1_max, x2_max)
    y2 = min(y1_max, y2_max)
    #print(x1,y1,x2,y2)
    intersect_width = max(0, x2 - x1)
    #print(f'intersect_width:{intersect_width}')
    intersect_height =  max(0, y2 - y1)
    #print(f'intersect_height:{intersect_height}')
    intersect_area = intersect_width*intersect_height
    #print(f'intersect_area:{intersect_area}')
    #compute the areas
    box1_area = (x1_max -x1_min)*(y1_max-y1_min)
    box2_area = (x2_max -x2_min)*(y2_max-y2_min)
    #union area
    union_area= box1_area + box2_area - intersect_area
    #print(f'union:{union_area}')
    if union_area <=0:
        return 0
    iou = intersect_area/union_area
    #print(f'iou:{iou}')
    return iou

def parse_boxes(list):
    parsed_boxes = []
    for i in list:
        cat_name = i['category']
        filename = i["file_name"]
        #confidence = i['conf_score']
        box = i['bbox']
        parsed_boxes.append((filename, cat_name, box))
    return parsed_boxes

def compute_ious(img_filename, ground_truth_data, predicted_data):
    boxes1 = parse_boxes(ground_truth_data)
    boxes2 = parse_boxes(predicted_data)
    results = []
    for items1 in boxes1:
        filename1, category1, box1 = items1
        if filename1 == img_filename:
            for items2 in boxes2:
                filename2, category2, box2 = items2
                if category1==category2 and filename1==filename2:
                    iou = calculate_iou(box1, box2)
                    results.append({
                        'file_name': filename1, 
                        'category_name': category1, 
                        'bbox1':box1,
                        'bbox2':box2,
                        'iou': iou})
    return results
img_filename = 'frame_000000.PNG'
ground_truth_data = bbox_actual_coco(folder_with_images, coco_annotations_file, 'result_folder')
predicted_data = bbox_coord_predict(image_folder_with_actual)
compute_ious(img_filename, ground_truth_data, predicted_data)


0: 384x640 (no detections), 3.8ms
Speed: 8.5ms preprocess, 3.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3.9ms
Speed: 1.0ms preprocess, 3.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3.8ms
Speed: 0.9ms preprocess, 3.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3.8ms
Speed: 0.9ms preprocess, 3.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3.8ms
Speed: 1.0ms preprocess, 3.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 3.8ms
Speed: 0.9ms preprocess, 3.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3.8ms
Speed: 0.9ms preprocess, 3.8ms inference, 0.3ms postprocess per image

[{'file_name': 'frame_000000.PNG',
  'category_name': 'person',
  'bbox1': [906.5, 372.1, 61.1, 156.64],
  'bbox2': [908, 376, 968, 535],
  'iou': 0.0},
 {'file_name': 'frame_000000.PNG',
  'category_name': 'person',
  'bbox1': [158.42, 412.53, 55.09, 114.48],
  'bbox2': [908, 376, 968, 535],
  'iou': 0.0},
 {'file_name': 'frame_000000.PNG',
  'category_name': 'person',
  'bbox1': [925.3, 471.92, 81.87, 149.78],
  'bbox2': [908, 376, 968, 535],
  'iou': 0.0},
 {'file_name': 'frame_000000.PNG',
  'category_name': 'person',
  'bbox1': [233.08, -1.22, 30.45, 41.28],
  'bbox2': [908, 376, 968, 535],
  'iou': 0.0},
 {'file_name': 'frame_000000.PNG',
  'category_name': 'person',
  'bbox1': [196.6, 0.75, 37.15, 46.81],
  'bbox2': [908, 376, 968, 535],
  'iou': 0.0}]